Задания 15.11

In [1]:
import numpy as np
import pandas as pd
from scipy import optimize
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, Lasso
import statsmodels.api as sm
from scipy import stats

#Номер1


In [2]:
def f(x, a):
    a0, a1, a2 = a
    return (a1 * x + a0) / (x + a2)

In [3]:
# Генерация значений x_i
x = np.arange(0.1, 3.1, 0.1)

In [8]:
print(epsilon_normal.shape)

(103,)


In [10]:
# Параметры
a = (0.0, 1.0, 1.0)
N = 30

# Генерация случайной погрешности ε_i для нормального распределения
epsilon_normal = np.random.normal(0, 0.05, N)

# Генерация значений y_i
y_normal =  f(x, a) + epsilon_normal

In [11]:
# Генерация случайной погрешности ε_i для равномерного распределения
epsilon_uniform = np.random.uniform(-0.05, 0.05, N)

# Генерация значений y_i
y_uniform = f(x, a) + epsilon_uniform

In [12]:
# Параметры распределения Лапласа
lambda_param = 1 / np.sqrt(2 * 0.05**2)  # Расчет λ, учитывая дисперсию

# Генерация случайной погрешности ε_i для распределения Лапласа
epsilon_laplace = np.random.laplace(0, 1/lambda_param, N)

# Генерация значений y_i
y_laplace = f(x, a) + epsilon_laplace

#Номер 2

In [13]:
# Используем OLS для регрессии
X = sm.add_constant(x)  # добавляем свободный член
model_ols = sm.OLS(y_normal, X).fit()
params_ols = model_ols.params
variance_ols = np.var(model_ols.params)  # дисперсия параметров

In [17]:
# Использование Lasso для ЛАР
model_lar = Lasso(alpha=0.1)  # можно настроить alpha
model_lar.fit(X, y_normal)
params_lar = np.array([model_lar.intercept_, model_lar.coef_[0], model_lar.coef_[1]])
variance_lar = np.var(params_lar)  # дисперсия параметров

In [15]:
def minimax_loss(params):
    y_pred = f(x, params)
    return np.sum(np.abs(y_normal - y_pred))

# Начальные приближения
starting_params = np.array(a)

# Оптимизация
result_minimax = optimize.minimize(minimax_loss, starting_params)
params_minimax = result_minimax.x
variance_minimax = np.var(params_minimax)  # дисперсия параметров

In [18]:
print("OLS Parameters:", params_ols)
print("OLS Variance:", variance_ols)

print("LAR Parameters:", params_lar)
print("LAR Variance:", variance_lar)

print("Minimax Parameters:", params_minimax)
print("Minimax Variance:", variance_minimax)

OLS Parameters: [0.26593766 0.18306868]
OLS Variance: 0.0017168167821397448
LAR Parameters: [0.47283421 0.         0.04958704]
LAR Variance: 0.045018803702832476
Minimax Parameters: [-0.00820698  0.90036966  0.71912828]
Minimax Variance: 0.15415294485591488


#ДЗ(номер3)


In [19]:
a = (0.0, 1.0, 1.0)
N = 30
num_simulations = 103  # Количество генераций
outliers_count = [1, 2, 3]  # Мы будем добавлять 1, 2 и 3 выброса в разные симуляции

# Счетчик случаев, когда выбросы остаются
outliers_stayed = {k: 0 for k in outliers_count}

for k in outliers_count:
    for _ in range(num_simulations):
        # Генерация значений x_i
        x = np.arange(0.1, 3.1, 0.1)

        # Генерация случайной погрешности ε_i для нормального распределения
        epsilon_normal = np.random.normal(0, 0.05, N)

        # Генерация значений y_i
        y_normal = f(x, a) + epsilon_normal

        # Вычисляем квантиль для добавления выбросов
        q95 = np.percentile(y_normal, 95)
        q99 = np.percentile(y_normal, 99)
        q999 = np.percentile(y_normal, 99.9)

        # Создаем выбросы
        outliers = np.random.choice([q95, q99, q999], k, replace=False)

        # Добавляем выбросы случайным образом в данные
        outlier_indices = np.random.choice(range(N), k, replace=False)
        y_with_outliers = np.copy(y_normal)
        y_with_outliers[outlier_indices] = outliers

        # OLS метод
        X = sm.add_constant(x)  # добавляем свободный член
        model_ols = sm.OLS(y_with_outliers, X).fit()

        # Проверяем, остались ли выбросы
        residuals = model_ols.resid
        if any(np.abs(residuals) > 3 * np.std(residuals)):  # Проверка на выбросы
            outliers_stayed[k] += 1

# Вывод результатов
for k in outliers_count:
    print(f'Количество выбросов: {k}, случаев, когда хотя бы один выброс остался: {outliers_stayed[k]} из {num_simulations}')


Количество выбросов: 1, случаев, когда хотя бы один выброс остался: 42 из 103
Количество выбросов: 2, случаев, когда хотя бы один выброс остался: 64 из 103
Количество выбросов: 3, случаев, когда хотя бы один выброс остался: 63 из 103
